# Setting up the API key

In [ ]:
import google.generativeai as genai
genai.configure(api_key='YOUR_API_KEY')

# Retrieving the vectorial database <br>and the documents

In [ ]:
import faiss
import os 
import json
import numpy as np
def embed_query(text):
    embedding_result = genai.embed_content(
        model='models/embedding-001',
        content=text,
        task_type='retrieval_query'
    )
    return np.array(embedding_result['embedding'], dtype=np.float32).reshape(1, -1)

def retriever(query, index, law_names, k=1):
    query_embedding = embed_query(query)
    faiss.normalize_L2(query_embedding)
    distances, indices = index.search(query_embedding, k)
    
    # Retrieve the top k law texts
    retrieved_laws = [law_names[indices[0][i]] for i in range(k)]
    
    return retrieved_laws

save_folder = "rag_system"
# Load index and mappings for retrieval
index = faiss.read_index(os.path.join(save_folder, "index.faiss"))
with open(os.path.join(save_folder, "documents.json"), "r") as f:
    docs_mapping = json.load(f)

# Extract law texts in the order of their keys
law_names = [law['law'] for law in docs_mapping.values()]
# Example query
query = "Freedom of speech rights for citizens"
# Retrieve the most relevant law
retrieved_law = retriever(query, index, law_names)
print("Retrieved Law:")
print(retrieved_law)


Retrieved Law:
['All citizens have the right to freedom of speech, provided it does not incite violence or spread falsehoods.']


# Without history

In [33]:
def create_prompt(query, retrieved_laws):
    prompt = (
        "Answer the following question based on the provided laws very briefly :\n\n"
        f"**Question:** {query}\n\n"
        "**Relevant Laws:**\n"
    )
    for i, law in enumerate(retrieved_laws, 1):
        prompt += f"{i}. {law}\n"
    prompt += (
        "\n"
        "Please provide a brief answer using the relevant laws above. "
        "If the laws do not directly address the question, explain based on the principles outlined in them."
    )
    return prompt

# Assume 'query' is the user's input and 'retrieved_laws' are from the retriever
def generate_response(query):
    model = genai.GenerativeModel('models/gemini-1.5-flash')  

    retrieved_laws = retriever(query, index, law_names, k=3)
    # Create the prompt
    prompt = create_prompt(query, retrieved_laws)
    # Generate response using the Gemini model
    response = model.generate_content(prompt)
    return response.text,retrieved_laws


query = "What are the rights of citizens regarding freedom of speech?"
response,retrieved_laws = generate_response(query)
# Print the model's response
print(response)

Citizens have the right to freedom of speech, but this right is limited; speech cannot incite violence or spread falsehoods.



# With history

In [ ]:
# Initialize conversation history
conversation_history = []
def add_message(role, content):
    """Add a message to the conversation history."""
    conversation_history.append({'role': role, 'content': content})
def create_prompt(query, retrieved_laws, history):
    """Create a prompt including the query, retrieved laws, and conversation history."""
    prompt = (
        "Answer the following question based on the provided laws and the conversation history:\n\n"
    )
    
    # Include conversation history
    for msg in history:
        prompt += f"**{msg['role'].capitalize()}:** {msg['content']}\n\n"
    
    prompt += (
        f"**Question:** {query}\n\n"
        "**Relevant Laws:**\n"
    )
    for i, law in enumerate(retrieved_laws, 1):
        prompt += f"{i}. {law}\n"
    prompt += (
        "\n"
        "Please provide a brief answer using the relevant laws and considering the previous conversation. "
        "If the laws do not directly address the question, explain based on the principles outlined in them."
    )
    return prompt
# Example usage:
model = genai.GenerativeModel('models/gemini-1.5-flash') 
# User's query
user_query = "What are the rights of citizens regarding freedom of speech?"
# Retrieve relevant laws
retrieved_laws = retriever(user_query, index, law_names, k=3)
# Create prompt with conversation history
prompt = create_prompt(user_query, retrieved_laws, conversation_history)
# Generate response
response = model.generate_content(prompt)
# Add user and assistant messages to history
add_message('user', user_query)
add_message('assistant', response.text)
# Next interaction
next_user_query = "How does this relate to peaceful assembly?"
# Create prompt with updated history
prompt = create_prompt(next_user_query, retrieved_laws, conversation_history)

# Generate response
next_response = model.generate_content(prompt)

# Add to history
add_message('user', next_user_query)
add_message('assistant', next_response.text)

# Continue the conversation as needed

In [29]:
conversation_history

[{'role': 'user',
  'content': 'What are the rights of citizens regarding freedom of speech?'},
 {'role': 'assistant',
  'content': 'Citizens have the right to freedom of speech, but this right is limited.  Speech cannot incite violence or spread falsehoods.  The principles of freedom of the press (law 3) also suggest a broad interpretation of freedom of expression, encompassing various forms of communication.\n'},
 {'role': 'user', 'content': 'How does this relate to peaceful assembly?'},
 {'role': 'assistant',
  'content': 'Freedom of speech (Law 1) and the right to peaceful assembly (Law 2) are closely related.  Peaceful assembly often involves the expression of views, which is protected under freedom of speech.  The principles of free expression, extended by Law 3 to encompass the press, further support the right to gather peacefully to communicate ideas and opinions.  However, both rights are conditional; speech and assembly must remain peaceful and not incite violence.\n'}]